## Create Base folders
**Date:** Oct 10, 2022

Este script genera los df que son la base para el webscrap. Crea un archivo por cada provincia, con la lista de ```idjudicatura```

In [1]:
import numpy as np
import pandas as pd
import regex
from unidecode import unidecode

import difflib

from pyprojroot import here

In [2]:
# Define Paths
raw = here()/'data/raw'
proc = here()/'data/proc'

***
### Load Data

Codigos de cantones del INEC - DPA

In [3]:
# Load data
dpa = pd.read_excel(raw/'inec/idcanton_2022.xlsx', sheet_name='PROVINCIAS', skiprows=1, usecols=['DPA_PROVIN', 'DPA_DESPRO'])
dpa = dpa.rename(columns={'DPA_PROVIN': 'idprov', 'DPA_DESPRO': 'provincia'})

Codigos de las judicaturas, enviadas por SATJE

In [4]:
# Codigos judicaturas
id_court = pd.read_excel(raw/'satje/0645_Id_Judicaturas.xlsx', skiprows=range(5))

# Drop tail observations and useless columns
id_court = id_court.loc[~id_court['JUDICATURA'].isna(), ['ID_JUDICATURA', 'JUDICATURA']]

Nombres de las judicaturas, by casos

In [5]:
# Nombres de judicaturas
courts_cases = pd.read_excel(raw/'satje/1123_Causas_delitos.xlsx',
    skiprows=range(6),
    usecols=['PROVINCIA', 'INSTANCIA', 'JUDICATURA', 'MATERIA', 'NOMBRE DELITO'])

# Delete useless rows
courts_cases = courts_cases.query('~INSTANCIA.isnull()').copy()

In [6]:
# Keep only names
courts_names = (courts_cases[['PROVINCIA', 'JUDICATURA']]
    .drop_duplicates(ignore_index=True)
)

Judicaturas Webscraped

In [7]:
# Load Data
idcourts_ws = pd.read_csv(raw/'satje/idjudicaturas_scraped.csv')

# Remove emprty columns
idcourts_ws = idcourts_ws.query('nombre!="No se encuentra"').copy()

# Clean text
idcourts_ws['nombre'] = idcourts_ws['nombre'].str.replace('Dependencia jurisdiccional: ', '')

***
### Get provinces

To get the province, I use the first 2 digits of the ID. And the data on Webscraped courts

In [8]:
# Province Code
idcourts_ws['idprov'] = idcourts_ws['id_dependencia'].astype(str).apply(lambda x: "0"+x[0] if len(x)<5 else x[0:2]) 
idcourts_ws['id_dependencia'] = idcourts_ws['id_dependencia'].astype(str).apply(lambda x: "0"+x if len(x)<5 else x) 

In [9]:
# Prepare Province-Codes data
dpa['idprov'] = dpa['idprov'].astype(str).apply(lambda x: x if len(x)==2 else "0"+x)
dpa['provincia'] = dpa['provincia'].map(unidecode)

In [10]:
# Merge data
idcourts_ws = pd.merge(
    idcourts_ws,
    dpa,
    how='left',
    on='idprov',
    validate='m:1',
    indicator=False
    )

***
### Create file with codes for ech province

Clean court name

In [11]:
idcourts_ws['nombre'] = idcourts_ws['nombre'].apply(lambda x: regex.sub('\n|\t', '', x))
idcourts_ws['judicatura'] = idcourts_ws['nombre'].apply(lambda x: x.split('Ciudad: ')[0])
idcourts_ws['ciudad'] = idcourts_ws['nombre'].apply(lambda x: x.split('Ciudad: ')[-1])

Create folders for each provincia

In [ ]:
# Folders for the estado
for idprov in np.unique(idcourts_ws['idprov']):

    # Subset df based on province
    provdf = idcourts_ws.loc[idcourts_ws['idprov']==idprov, ['idprov', 'provincia', 'id_dependencia']]

    # Add estado
    provdf['estado'] = np.nan

    # Create folder for provincia
    try:
        (proc/f'estado/prov{idprov}').mkdir(parents=True)
    except FileExistsError:
        pass

    # Save file
    provdf.to_parquet(proc/f'estado/prov{idprov}/courts_status.parquet')

***
## Define crime list

In [7]:
delitos = pd.read_csv(raw/'lista_delitos.csv')

In [12]:
delitos = pd.read_csv(raw/'lista_delitos_15_20.csv')

In [14]:
list(delitos['NOMBRE DELITO'])

['141 FEMICIDIO',
 '144 HOMICIDIO',
 '152 LESIONES, NUM. 3',
 '170 ABUSO SEXUAL, INC.3',
 '171 VIOLACION, INC.1',
 '171 VIOLACION, MAXIMA PENA, INC.2 ,NUM. 4',
 '171 VIOLACION, MAXIMA PENA,INC.2 , NUM. 2',
 '171 VIOLACION, MAXIMA PENA,INC.2 , NUM. 3',
 '171 VIOLACION, NUM. 2',
 '189 ROBO, INC.1',
 '196 HURTO, INC.1',
 '159 VIOLENCIA CONTRA LA MUJER O MIEMBROS DEL NUCLEO FAMILIAR',
 '209 CONTRAVENCION DE HURTO',
 '249 MALTRATO O MUERTE DE MASCOTAS O ANIMALES DE COMPAÑIA, 1',
 '393 CONTRAVENCIONES DE PRIMERA CLASE, INC.1, NUM. 1',
 '396 CONTRAVENCIONES DE CUARTA CLASE, INC.1, NUM. 1',
 '396 CONTRAVENCIONES DE CUARTA CLASE, INC.1, NUM. 2',
 '396 CONTRAVENCIONES DE CUARTA CLASE, INC.1, NUM. 4',
 'DAÑOS Y PERJUICIOS',
 'DEFECTOS Y VICIOS OCULTOS',
 'ABUSO DE CONFIANZA',
 'AMPARO DE LIBERTAD',
 'ASESINATO',
 'ATENTADO CONTRA EL PUDOR, VIOLACION Y ESTUPRO',
 'CONFLICTO DE COMPETENCIA',
 'CONTRA LA VIDA',
 'DELITOS ADUANEROS',
 'DELITOS CONTRA LA ADMINISTRACION PUBLICA',
 'DELITOS DE TRANSITO'